In [59]:
import pandas as pd 
import matplotlib.pyplot as pyplot
import seaborn as sns
import warnings

%matplotlib inline
warnings.filterwarnings("ignore")

In [60]:
dataset = pd.read_csv('./transactions.csv', on_bad_lines='skip')
dataset

,TransactionID,CustomerID,Date,Amount,Currency,TransactionType
0,1001,CUST001,2025-07-01,5000.00,USD,Deposit
1,1002,CUST001,2025-07-01,4500.00,USD,Deposit
2,1003,CUST002,2025-07-01,8000.00,USD,Deposit
3,1004,CUST003,2025-07-02,9500.00,USD,Deposit
4,1005,CUST001,2025-07-02,3000.00,USD,Deposit
...,...,...,...,...,...,...
4993,5996,CUST085,2025-08-30,4190.42,USD,Deposit
4994,5997,CUST076,2025-11-15,3414.47,USD,Withdrawal
4995,5998,CUST077,2025-12-16,16801.83,USD,Deposit
4996,5999,CUST091,2025-12-02,12830.83,USD,Deposit


In [61]:
# Dataset information
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4998 entries, 0 to 4997
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   TransactionID    4998 non-null   int64  
 1   CustomerID       4998 non-null   object 
 2   Date             4998 non-null   object 
 3   Amount           4998 non-null   float64
 4   Currency         4998 non-null   object 
 5   TransactionType  4998 non-null   object 
dtypes: float64(1), int64(1), object(4)
memory usage: 234.4+ KB


In [62]:
# For each customer, calculate the total deposit amonut per day
deposits_per_day = dataset.groupby(['CustomerID', 'Date']).agg(
    total_amount=('Amount', sum)
)

deposits_per_day


total_amount
CustomerID Date                    
CUST001    2025-07-01      10489.18
           2025-07-02       9900.00
           2025-07-08       2200.00
           2025-07-09      18420.16
           2025-07-21       2102.74
...                             ...
CUST100    2025-12-13       3031.13
           2025-12-19       7554.06
           2025-12-26      12342.39
           2025-12-27       4627.44
           2025-12-30      19918.87

[4359 rows x 1 columns]

In [63]:
# Identify customers who made multiple deposits on the same day that sum to just under $10,000

# Create a dataframe with amounts less than or equal to just under 10k
less_than_10k = dataset.loc[dataset['Amount'] <= 9999]

# Group by CustomerID and Date, then get sum and count for deposits
summed_amounts = less_than_10k.groupby(['CustomerID', 'Date']).agg(
    total_amount=('Amount', 'sum'),
    deposit_count=('Amount', 'count')
)

# Get suspicious customers
suspicious_customers = summed_amounts[(summed_amounts['total_amount'] < 10000) & (summed_amounts['deposit_count'] > 1)]

# Dispaly
suspicious_customers

total_amount  deposit_count
CustomerID Date                                   
CUST001    2025-07-02       9900.00              3
           2025-08-19       5593.83              2
CUST003    2025-08-20       6231.16              2
           2025-08-24       3712.41              2
CUST005    2025-07-03       9900.00              4
...                             ...            ...
CUST094    2025-12-08       8048.85              2
CUST099    2025-09-29       9001.14              2
           2025-10-03       9915.36              2
CUST100    2025-08-13       7011.18              2
           2025-12-27       4627.44              2

[70 rows x 2 columns]

In [64]:
# Flag these customers and their suspicious transactions for further review.
suspicious_transactions = suspicious_customers.index.to_frame(index=False)

# Merge with original dataset
flagged_transactions = dataset.merge(suspicious_transactions, on=['CustomerID', 'Date'])

# Create Flagged column
flagged_transactions['Flagged'] = 'Yes'
flagged_transactions

,TransactionID,CustomerID,Date,Amount,Currency,TransactionType,Flagged
0,1005,CUST001,2025-07-02,3000.00,USD,Deposit,Yes
1,1006,CUST001,2025-07-02,3500.00,USD,Deposit,Yes
2,1007,CUST001,2025-07-02,3400.00,USD,Deposit,Yes
3,1009,CUST005,2025-07-03,2000.00,USD,Deposit,Yes
4,1010,CUST005,2025-07-03,2500.00,USD,Deposit,Yes
...,...,...,...,...,...,...,...
148,5756,CUST012,2025-07-04,2868.94,USD,Withdrawal,Yes
149,5950,CUST022,2025-10-14,2046.37,USD,Withdrawal,Yes
150,5961,CUST084,2025-08-23,5314.90,USD,Deposit,Yes
151,5972,CUST055,2025-08-31,2085.11,USD,Withdrawal,Yes
